<a href="https://colab.research.google.com/github/Abi0711/LLM_genre_classification/blob/main/LLM_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers datasets ipywidgets
!pip install transformers
# !pip install transformers[torch]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 16.5 MB/s eta 0:00:00


In [2]:
import torch
from torch.utils.data import DataLoader
from sklearn.metrics import f1_score
from tqdm.notebook import tqdm
from datasets import load_dataset
from transformers import DistilBertTokenizerFast
from transformers import DistilBertForSequenceClassification
import numpy as np
from pprint import pprint
import os
import json
import nltk
import string
import torch.nn as nn
import pandas as pd
from bisect import bisect_right
from collections import Counter
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from tqdm import tqdm
from datasets import Dataset
import torch.optim as optim
from sklearn.model_selection import KFold
from scipy.stats import uniform
import random

In [3]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
# path = '/content/drive/MyDrive/'

# Load the training data
train_data = json.load(open(os.path.join(path,"data", "genre_train.json"), "r"))
X = train_data['X']
Y = train_data['Y'] # id mapping: 0 - Horror, 1 - Science Fiction, 2 - Humour, 3 - Crime Fiction
docid = train_data['docid'] # these are the ids of the books which each training example came from

# Load the test data
# the test data does not have labels
test_data = json.load(open(os.path.join(path,"data", "genre_test.json"), "r"))
Xt = test_data['X']
combined_train = {
    'label': Y,
    'text': X
}
data = Dataset.from_dict(combined_train)

In [7]:
from transformers import AutoTokenizer, DebertaForSequenceClassification

model_name = "distilbert-base-uncased"
tok = DistilBertTokenizerFast.from_pretrained(model_name)


tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

In [9]:
max_length = 512
def tokenize_fn(X):
    return tok(X["text"], max_length = max_length, truncation = True, padding="max_length")
shuffle_train_data = data.shuffle()
tokenized_train_data = shuffle_train_data.map(tokenize_fn, batched=True)
tokenized_train_data = tokenized_train_data.with_format("torch", columns=["label", "input_ids", "attention_mask"])

Map:   0%|          | 0/6743 [00:00<?, ? examples/s]

In [10]:
pprint(tokenized_train_data[range(0,5)])

{'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]]),
 'input_ids': tensor([[  101,  1000,  2079,  ...,     0,     0,     0],
        [  101,  2356,  8471,  ...,     0,     0,     0],
        [  101, 22715,  2483,  ...,     0,     0,     0],
        [  101,  1998,  2059,  ...,     0,     0,     0],
        [  101,  2009,  2790,  ...,     0,     0,     0]]),
 'label': tensor([1, 0, 1, 1, 2])}


In [ ]:
# model.parameters

In [ ]:
def finetune_model(model, dataset_train, batch_size=8, n_epochs=2, learning_rate=1e-5):
    model.train(True)

    # Create a pytorch data loaders to make it easy to iterate over batches of training data
    dataloader = DataLoader(dataset_train, batch_size=batch_size, shuffle=True)

    n_batches = (len(dataset_train) - 1) // batch_size + 1
    # Add weight decay to W and not biases
    no_decay = ['bias', 'LayerNorm.weight']
    optimizer_grouped_parameters = [
        {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
        {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
    ]
    # Use AdamW optimizer
    optimiser = torch.optim.AdamW(params = optimizer_grouped_parameters, lr=learning_rate)

    # run the training loop
    print(f'{"Epoch":>10} {"Batch":>20} {"Loss":>10}')
    for epoch in range(n_epochs):
        running_loss = 0
        for (b, batch) in enumerate(tqdm(dataloader)):

            # run the transformer forwards
            outputs = model(
                labels = batch["label"].to(device),
                input_ids = batch["input_ids"].to(device),
                attention_mask = batch["attention_mask"].to(device),
            )
            # get the classification loss
            loss = outputs.loss

            # backpropagate then apply the optimiser
            optimiser.zero_grad()
            loss.backward()
            optimiser.step()
            running_loss +=loss.cpu().item()

            # print the loss
        print(f'{epoch+1:>10} {f"{b+1} / {n_batches}":>20} {format(running_loss/n_batches, ".3f"):>10}')
        running_loss = 0

    return model

In [ ]:
def evaluate_model(model, dataset_val, batch_size=8):
    model.train(False)
    f1 = 0
    # create a pytorch data loaders to make it easy to iterate over batches of validation data
    dataloader = DataLoader(dataset_val, batch_size=batch_size, shuffle=False)

    y_true = []
    y_pred = []
    with torch.no_grad():
        for batch in tqdm(dataloader):
            y_true.append(batch["label"].numpy())

            # run the transformer forwards
            outputs = model(
                input_ids = batch["input_ids"].to(device),
                attention_mask = batch["attention_mask"].to(device),
            )

            logits = outputs.logits.cpu().numpy()  # (batch_size, num_labels)
            y_pred.append(logits[:, 1] > 0)

    f1 = f1_score(np.concatenate(y_true), np.concatenate(y_pred), average = "macro")
    model.train(True)
    return f1

In [ ]:
# Change device to what is available
if torch.cuda.is_available():
    device = "cuda"
elif torch.backends.mps.is_available():
    device = "mps"
else:
    device = "cpu"

In [ ]:
# Perform k-fold cross-validation
k_folds = 5
kf = KFold(n_splits=k_folds)

best_performance = 0.0
for i in range(3):
    print(i)
    lr = random.uniform(1e-6, 1e-4)
    batch = 8
    epoch = random.randrange(8,15)
    total_f1 = 0
    for train_index, val_index in kf.split((X)):
        # Split your data into training and validation based on the fold
        train_data = tokenized_train_data.select(train_index)
        val_data = tokenized_train_data.select(val_index)
        # Finetune the model on the training data
        model = DistilBertForSequenceClassification.from_pretrained(model_name, num_labels=4)
        model.to(device)
        print(lr, batch, epoch)
        current_model = finetune_model(model, train_data, batch_size=batch,learning_rate=lr,  n_epochs=epoch)

        # Evaluate the model on the validation data and calculate performance

        total_f1 += evaluate_model(current_model, val_data)

    # Calculate the average performance for this hyperparameter combination
    avg_performance = total_f1 / k_folds

    # Check if this combination is the best so far
    if avg_performance > best_performance:
        best_performance = avg_performance
        best_hyperparameters = (lr, batch, epoch)
    print("Best: ",best_hyperparameters)
    print(best_performance)


0


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.bias', 'classifier.bias', 'pre_classifier.weight', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


3.046620489925351e-05 8 9
     Epoch                Batch       Loss


100%|██████████| 675/675 [01:09<00:00,  9.78it/s]


         1            675 / 675      0.473


...

        13            675 / 675      0.017


100%|██████████| 675/675 [01:05<00:00, 10.31it/s]


        14            675 / 675      0.021


100%|██████████| 169/169 [00:05<00:00, 32.66it/s]

Best:  (3.046620489925351e-05, 8, 9)
0.29962989957582853


In [ ]:
# Now train a new model with all train data on the best hyperparameters
best_hyperparameters
model = DistilBertForSequenceClassification.from_pretrained(model_name, num_labels=4)
model.to(device)
current_model = finetune_model(model, tokenized_train_data, batch_size=8,learning_rate=best_hyperparameters[0],  n_epochs=10)


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.bias', 'classifier.bias', 'pre_classifier.weight', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


     Epoch                Batch       Loss


100%|██████████| 843/843 [01:21<00:00, 10.30it/s]


         1            843 / 843      0.436


100%|██████████| 843/843 [01:21<00:00, 10.31it/s]


...

         8            843 / 843      0.013


100%|██████████| 843/843 [01:21<00:00, 10.30it/s]


         9            843 / 843      0.017


100%|██████████| 843/843 [01:21<00:00, 10.31it/s]

        10            843 / 843      0.010


In [ ]:
testdict = {
    'text': Xt
}
# Create test data
test = Dataset.from_dict(testdict)
tokenized_test_data = test.map(tokenize_fn, batched=True)
test_data = tokenized_test_data.with_format("torch", columns=["input_ids", "attention_mask"])

Map:   0%|          | 0/47183 [00:00<?, ? examples/s]

In [ ]:

def predict_model(model, test, batch_size = 8):
    model.train(False)
    dataloader = DataLoader(test, batch_size=batch_size)
    pred = []
    with torch.no_grad():

        for (b, batch) in enumerate(tqdm(dataloader)):
            outputs = model(
                  input_ids = batch["input_ids"].to(device),
                  attention_mask = batch["attention_mask"].to(device),
              )
            logits = outputs.logits
            # print(logits)
            for i in logits:
                predicted_class_id = i.argmax().item()
                # print(predicted_class_id)
                pred.append(predicted_class_id)
    return pred


# predict on the test data
Y_test_pred = predict_model(current_model, test_data)

# write out the csv file
# first column is the id, it is the index to the list of test examples
# second column is the prediction as an integer
fout = open(path+"/out.csv", "w")
fout.write("Id,Predicted\n")
for i, line in enumerate(Y_test_pred): # Y_test_pred is in the same order as the test data
    fout.write("%d,%d\n" % (i, line))
fout.close()
print("done")

100%|██████████| 5898/5898 [03:02<00:00, 32.40it/s]


done
